In [1]:
import simuOpt
simuOpt.setOptions(alleleType='short', quiet=True, numThreads=4)
import simuPOP as sim
import numpy as np
import pandas as pd
import random
import h5py
from saegus import analyze, operators, parameters, parse, breed
np.set_printoptions(suppress=True, precision=5)

## Data Storage

In [2]:
data_file = h5py.File("3_selection_data_example.hdf5")

In [3]:
data_file.create_group('allele')

<HDF5 group "/allele" (0 members)>

In [4]:
data_file['allele'].attrs['columns'] = list(map(np.string_, ['locus', 'alpha', 'omega', 'minor', 'major']))

In [5]:
example_pop = sim.loadPopulation('example_pop.pop')
example_pop.addInfoFields(['ind_id', 'mother_id', 'father_id', 'g', 'p', 'sample', 'selection'])
example_pop.dvars().gen = 0
sim.tagID(example_pop)
sim.stat(example_pop, numOfSegSites=sim.ALL_AVAIL, 
         vars=['numOfSegSites','segSites', 'fixedSites'])
sim.stat(example_pop, alleleFreq=sim.ALL_AVAIL)
segregating_loci = example_pop.dvars().segSites

In [6]:
example_pop.dvars().gen

0

In [7]:
tf = parse.TusonFounders()

In [8]:
recom_map = tf.parse_recombination_rates('genetic_map.txt')

In [9]:
allele_states = analyze.gather_allele_data(example_pop)
allele_frequencies = analyze.gather_allele_frequencies(example_pop, allele_states)
gwas = analyze.GWAS(example_pop, np.array(segregating_loci, dtype=np.int_), allele_states[:, 3], 'example')

In [10]:
try:
    sim.stat(example_pop, alleleFreq=sim.ALL_AVAIL, subPops=[(0, 0), (0, 1)],
       vars=['alleleFreq_sp'])
except ValueError:
    sim.stat(example_pop, alleleFreq=sim.ALL_AVAIL)

In [11]:
trait = parameters.Trait()

In [12]:
qtl = random.sample(segregating_loci, 30)

In [13]:
ae_table = trait.construct_allele_effects_table(example_pop, qtl, random.expovariate, 1)

In [14]:
ae_array = trait.construct_ae_array(ae_table, qtl)

In [15]:
operators.calculate_g(example_pop, ae_array)

In [16]:
operators.calculate_error_variance(example_pop, 0.7)

In [17]:
example_pop.dvars().epsilon

21.707043724116794

In [18]:
data_file.create_group('trait')

<HDF5 group "/trait" (0 members)>

In [19]:
data_file['trait'].attrs['error_variance'] = example_pop.dvars().epsilon

In [20]:
operators.calculate_p(example_pop)

In [21]:
example_pop.sortIndividuals('p')
example_pop.setVirtualSplitter(sim.ProportionSplitter([0.95, 0.05], names=['not_selected', 'selected']))

In [22]:
example_pop.indInfo('p', [0, 1])

(87.90492164407843,
 88.97960410930884,
 92.22092571243546,
 96.60481594346916,
 105.80015063080535)

In [23]:
for ind in example_pop.individuals([0, 1]):
    ind.setSex(1)
for ind in example_pop.individuals([0, 0]):
    ind.setSex(2)

In [24]:
example_pop.dvars().gen

0

In [25]:
minor_alleles = allele_states[:, 3]

In [ ]:
allele_frequencies[:, 3]

In [ ]:
segregating_loci

In [ ]:
example_pop.numVirtualSubPop()

In [ ]:
sim.stat(example_pop, 
         alleleFreq=example_pop.dvars().segSites, subPops=[(0, 0)], vars=['alleleFreq_sp'])

In [ ]:
example_pop.dvars().alleleFreq

In [ ]:
example_pop.dvars([0, 0]).alleleFreq

In [ ]:
np.array(data_file['allele/frq/0'])

In [26]:
qtl = np.array(sorted(qtl))

In [28]:
example_pop.evolve(
    preOps=[
        operators.SampleSetter(50),
        sim.Stat(alleleFreq=segregating_loci, subPops=[(0, 0)],
                vars=['alleleFreq_sp']),
        operators.GenoAdditiveArray(qtl, ae_array),
        operators.PhenoAdditive(),
        operators.HDF5MinorAlleleFrequencies(data_file, 'allele/frq', 
                                            minor_alleles, [0, 0]),
        operators.HDF5Trait(data_file, 'trait/p', 'p',  [0, 0]),
        operators.HDF5Trait(data_file, 'trait/g', 'g', [0, 0]),
        operators.PreSelection([0.95, 0.05]),
    ],
    matingScheme=
    sim.HomoMating(
        sim.PolyParentsChooser(
            polySex=sim.MALE, polyNum=10
            ),
        sim.OffspringGenerator(ops=[
            sim.IdTagger(),
            sim.PedigreeTagger(),
            sim.Recombinator(rates=recom_map),
        ]), subPopSize=1000
    ),
    finalOps=[        
        operators.SampleSetter(50),
        sim.Stat(alleleFreq=segregating_loci, subPops=[(0, 0)],
                vars=['alleleFreq_sp']),
        operators.GenoAdditiveArray(qtl, ae_array),
        operators.PhenoAdditive(),
        operators.HDF5MinorAlleleFrequencies(data_file, 'allele/frq', 
                                            minor_alleles, [0, 0]),
        operators.HDF5Trait(data_file, 'trait/p', 'p',  [0, 0]),
        operators.HDF5Trait(data_file, 'trait/g', 'g', [0, 0]),
    ],
    gen=2
)

2

In [29]:
g0 = np.array(data_file['trait/p/0'])

In [30]:
g1 = np.array(data_file['trait/p/1'])

In [31]:
g2 = np.array(data_file['trait/p/2'])

In [32]:
np.mean(g0)

62.754556380473005

In [33]:
np.mean(g1)

65.974680193754452

In [34]:
np.mean(g2)

63.042270066959212

In [35]:
data_file.close()

In [ ]:
np.array(data_file['trait/g/1'])

In [ ]:
example_pop.dvars().gen

In [ ]:
np.array(data_file['trait/g/0'])

In [ ]:
np.array(data_file['trait/0/p'])

In [ ]:
np.array(data_file['trait/1/p'])

In [ ]:
example_pop.numVirtualSubPop()

In [ ]:
example_pop.popSize()

In [ ]:
example_pop.indInfo('p')

In [ ]:
pedigree = np.array([example_pop.indInfo('father_id'),
          example_pop.indInfo('mother_id'),
          example_pop.indInfo('ind_id')]).T

In [ ]:
pedigree

In [ ]:
print(pedigree)

In [ ]:
operators.calculate_g(example_pop, ae_array)

In [ ]:
operators.calculate_p(example_pop)

In [ ]:
example_pop.indInfo('p')

In [ ]:
example_pop.sortIndividuals('p')
example_pop.setVirtualSplitter(sim.ProportionSplitter([0.95, 0.05], names=['not_selected', 'selected']))
for ind in example_pop.individuals([0, 1]):
    ind.setSex(1)
for ind in example_pop.individuals([0, 0]):
    ind.setSex(2)

In [ ]:
example_pop.indInfo('p', [0, 1])

In [ ]:
for ind in example_pop.individuals([0, 1]):
    print(ind.sex())

## Determine Sample

In [ ]:
sampled_inds = random.sample(example_pop.indInfo('ind_id'), 50)
for ind in sampled_inds:
    example_pop.indByID(ind).sample = 1

In [ ]:
example

In [ ]:
ped.locateRelatives(sim.COMMON_OFFSPRING)